<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NB15__ML_AutoML__H2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b><i>AUTOML - H2O</i></b></h1></center>


# O QUE É AUTOML?

> AUTOML significa _Automated Machine Learning_, referenciando-se ao processo de automatizar o desenvolvimento dos modelos de _Machine Learning_. Como sabemos, desenvolver modelos preditivos de _Machine Learning_ envolve conhecimentos avançados dos algoritmos, linguagem de programação, Estatística e etc, além do considerável tempo para desenvolvimento. Com AutoML é possível acelerar o processo de desenvolvimento dos projetos relacionados à _Machine Learning_.

A seguir, algumas características do AutoML:
* Implementar soluções de _Machine Learning_ sem extensivo conhecimento de programação em linguagens como Python, GO, Spark e etc.;
* Economia de tempo e recursos;
* Desenvolvimento rápido;
* Fácil de usar, pois a sintaxe dos comandos é muito parecida com o Pandas;
* Muitos [algoritmos](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science.html).

![AutoML](https://raw.githubusercontent.com/MathMachado/Materials/master/AutoML_Cycle.png)

# O QUE É H2O?

> H2O é um AutoML produzido pela H2O.ai, empresa que tem por objetivo tornar _Machine Learning_ acessível à todos. A seguir, algumas razões para considerar h2o:

1. A empresa tem hoje uma comunidade de Cientistas de Dados que conta com mais de 129.000 membros e atende mais da metade das empresas da Fortune 500;
2. A empresa divulgou que houve um aumento de usuários na ordem dos 330% nos últimos 2 anos!
3. O AutoML H2O suporta os algoritmos de Machine Learning mais usados para AutoML.
4. Há Kaggle Grandmaster usando H2O nas competições do [Kaggle](https://www.kaggle.com/) e o melhor de tudo é que as soluções usando H2O estão bem ranqueadas no _leaderboard_. Isso significa que o H2O fornece ótimos resultados/modelos!
5. A empresa H2O.ai está no quadrante visionário do Garther para Ciência de Dados e Machine Learning, como você pode ver na figura adiante:

![](https://raw.githubusercontent.com/MathMachado/Materials/master/Gartner_DataScienceTools.png)

Fonte: [Gartner’s 2020 Magic Quadrant For Data Science And Machine Learning Platforms Has Many Surprises](https://www.forbes.com/sites/janakirammsv/2020/02/20/gartners-2020-magic-quadrant-for-data-science-and-machine-learning-platforms-has-many-surprises/#3d8e24e03f55).

# REFERÊNCIAS

* [A Deep dive into H2O’s AutoML](https://towardsdatascience.com/a-deep-dive-into-h2os-automl-4b1fe51d3f3e);
* [Gentle Introduction to AutoML from H2O.ai](https://medium.com/analytics-vidhya/gentle-introduction-to-automl-from-h2o-ai-a42b393b4ba2)
* [Introduction to H2O.ai](https://medium.com/@jamal.robinson/introduction-to-h2o-ai-1ba51a884f02);
* [Democratising Machine learning with H2O](https://towardsdatascience.com/democratising-machine-learning-with-h2o-7f2f79e10e3f).

[**Python**] - Instalar o h2o:

In [ ]:
!pip install h2o

# **EXEMPLO 1 - Estimar o tipo do vinho**

> Nesta aplicação, vamos usar o dataframe [wine quality](https://archive.ics.uci.edu/ml/datasets/wine+quality) extraído do repositório da UCI Machine Learning Repository. Nosso objetivo é prever o tipo do vinho baseado nas suas propriedades químicas do produto.



[**Python**] - Carregar h2o:

In [ ]:
import h2o
from h2o.automl import H2OAutoML

from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

[**Python**] - Inicializar o h2o:

In [ ]:
h2o.init()

[**Python**] - Carregar os dados

> Observe abaixo que a forma de carregar os dados é semelhante ao Pandas. No entanto, vale lembrar que os dados estão armazenados na memória como um H2OFrame.

### 0. Carregar bibliotecas do Python

[**Python**] - Importar as bibliotecas necessárias:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf

from tensorflow import keras

[**Python**] - Verificar a versão do Tensorflow
> Assegurar que está a utilizar a versão 2.x.

In [ ]:
tf.__version__

[**Python**] - Definir o número de casas decimais

In [ ]:
np.set_printoptions(precision= 3)

### 1. Carregar os dados

[**Python**] - Carregar os dados:

In [ ]:
df_Red= pd.read_table("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
df_Red.head()

In [ ]:
df_White= pd.read_table('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep= ';')
df_White.head()

[**Python**] - Mostrar o número de linhas e colunas de cada dataframe:

In [ ]:
print(f'Dimensão de df_Red: {df_Red.shape}; Dimensão de df_White: {df_White.shape}')

[**Python**] - Construir a variável 'type_wine' (tipo do vinho):

In [ ]:
df_Red['type_wine']= 0
df_White['type_wine']= 1

[**Python**] - Empilhar os dois dataframes: df_Red e df_White:

In [ ]:
df_Wine= pd.concat([df_Red, df_White], ignore_index=True)
df_Wine.head()

In [ ]:
df_Wine.tail()

[**Python**] - Mostrar o número de linhas e colunas do dataframe df_Wine:

In [ ]:
df_Wine.shape

### 2. Pré-processamento e transformação dos dados

[**Python**] - Renomear o nome das colunas usando letras minúsculas:

In [ ]:
df_Wine.columns = df_Wine.columns.str.strip().str.lower().str.replace(' ', '_')

In [ ]:
df_Wine.head()

[**Python**] - Passar o dataframe df_Wine ao H2O:

In [ ]:
Wine= h2o.H2OFrame(df_Wine)
Wine.head()

Este é um problema de classificação. Portanto, devemos definir a variável-target como uma classe.

[**Python**] - Definir a variável-target como classe:

In [ ]:
Wine['type_wine']= Wine['type_wine'].asfactor()

[**Python**] - Definir as amostras de treinamento e validação:

In [ ]:
# Selecionar aleatoriamente 90% dos dados para a amostra de treinamento e 10% para a amostra de validação:
Wine_Treinamento, Wine_Teste= Wine.split_frame(ratios=[0.9], seed= 20111974)

[**Python**] - Mostrar os tamanhos das amostras de validação e treinamento:

In [ ]:
print(Wine_Treinamento.shape, Wine_Teste.shape)

[**Python**] - Definir as variáveis preditoras:

In [ ]:
preditoras= list(Wine_Treinamento.columns)
target= "type_wine" 
preditoras.remove(target)
preditoras

[**Python**] - Vamos começar com o modelo queridinho dos Kaggle Grandmasters: GBM (Gradient Boosting Machines):

In [ ]:
gbm= H2OGradientBoostingEstimator()
gbm.train(x= preditoras, y= target, training_frame= Wine_Treinamento)

[**Python**] - Vamos ver o resultado (que modelo o algoritmo nos recomenda):

In [ ]:
print(gbm)

Muitas métricas para avaliarmos a performance do modelo. Foco nas métricas a seguir:

[**Python**] - Métricas para avaliar a performance do GBM:

In [ ]:
perf= gbm.model_performance(Wine_Teste)
print(perf)

[**Python**] - Configurar o AutoML:

In [ ]:
h2o_Wine= H2OAutoML(max_models= 10, max_runtime_secs = 300, seed = 20111974)
h2o_Wine.train(x= preditoras, y= target, training_frame= Wine_Treinamento)

[**Python**] - Mostrar o leaderborder, que é a tabela com os modelos ajustados ao dataframe:

In [ ]:
Leader_Board= h2o_Wine.leaderboard
Leader_Board.head()

Como você pode ver no Leaderboarder, o AutoML ajustou vários modelos, incluindo GBM e XGBoost. De fato, GBM é o melhor modelo para este dataframe.

[**Python**] - Mostrar a performance do modelo na amostra de validação:

In [ ]:
h2o_Wine.leader.model_performance(Wine_Teste)

[**Python**] - Predições com o modelo vencedor

In [ ]:
Y_Predito= h2o_Wine.leader.predict(Wine_Teste)
Y_Predito

[**Python**] - Compare os resultados:

In [ ]:
Wine_Teste

[**Python**] - Mostrar todos os parâmetros do modelo campeão:

In [ ]:
ID_Modelo_Campeao= h2o_Wine.leader.model_id
ID_Modelo_Campeao

In [ ]:
Modelo_Parametros= h2o.get_model(ID_Modelo_Campeao)
Modelo_Parametros.params

[**Python**] - Desligar o cluster h2o:

In [ ]:
h2o.cluster().shutdown()

# **EXEMPLO 2: Distinguir cédulas verdadeiras das falsas**

O exemplo a seguir foi extraído do site [OpenML](https://www.openml.org/home). Este é um problema interessante, que é o de distinguir cédulas verdadeiras de notas falsas. Os dados foram extraídos de imagens tiradas de cédulas verdadeiras e falsas. Para digitalização, foi usada uma câmera industrial normalmente usada para inspeção de impressão. As imagens finais têm 400x 400 pixels. Devido à lente do objeto e à distância do objeto investigado, foram obtidas imagens em escala de cinza com uma resolução de cerca de 660 dpi. Uma ferramenta Wavelet Transform foi usada para extrair recursos dessas imagens.

* Este é o endereço do dataframe: https://www.openml.org/d/1462;
* Descrição das variáveis - [banknote authentication Data Set](https://archive.ics.uci.edu/ml/datasets/banknote+authentication).


[**Python**] - Carregar h2o:

In [ ]:
import h2o
from h2o.automl import H2OAutoML

[**Python**] - Inicializar o h2o:

In [ ]:
h2o.init()

[**Python**] - Carregar os dados

In [ ]:
Notas= h2o.import_file('https://raw.githubusercontent.com/MathMachado/DataFrames/master/Banknote-authentication-dataset.csv')
Notas.summary()

Este é um problema de classificação. Portanto, devemos definir a variável-target como uma classe.

[**Python**] - Definir a variável-target como classe:

In [ ]:
Notas['Class']= Notas['Class'].asfactor()

[**Python**] - Definir as amostras de treinamento e validação dos modelos preditivos:

In [ ]:
Notas_Treinamento, Notas_Teste= Notas.split_frame(ratios=[0.9], seed= 20111974)

[**Python**] - Mostrar os tamanhos das amostras de validação e treinamento:

In [ ]:
print(Notas_Treinamento.shape, Notas_Teste.shape)

[**Python**] - Definir as variáveis preditoras e target:

In [ ]:
preditoras= list(Notas_Treinamento.columns)
target= "Class"
preditoras.remove(target)
preditoras

[**Python**] - Configuração do AutoML:

In [ ]:
h2o_Notas = H2OAutoML(max_models= 10, max_runtime_secs = 300, seed= 20111974)
h2o_Notas.train(x= preditoras, y= target, training_frame= Notas_Treinamento)

[**Python**] - Listar os modelos candidatos:

In [ ]:
Leader_Board= h2o_Notas.leaderboard
Leader_Board.head()

[**Python**] - Mostrar a performance do modelo na amostra de treinamento:

In [ ]:
h2o_Notas.leader.model_performance(Notas_Treinamento)

[**Python**] - Mostrar a performance do modelo na amostra de validação

In [ ]:
h2o_Notas.leader.model_performance(Notas_Teste)

[**Python**] - Fazer predições/estimativas com o modelo campeão:

In [ ]:
Y_Predito= h2o_Notas.leader.predict(Notas_Teste)
Y_Predito

[**Python**] - Mostrar todos os parâmetros do modelo campeão:

In [ ]:
ID_Modelo_Campeao= h2o_Notas.leader.model_id
ID_Modelo_Campeao

In [ ]:
Modelo_Parametros= h2o.get_model(ID_Modelo_Campeao)
Modelo_Parametros.params

[**Python**] - Desligar o cluster h2o:

In [ ]:
h2o.cluster().shutdown()

# **EXEMPLO 3 - Estimar a espécie de flores (Iris dataframe)**

![Iris](https://raw.githubusercontent.com/MathMachado/Materials/master/Iris.png)

Fonte: [Data Science Example - Iris dataset](http://www.lac.inpe.br/~rafael.santos/Docs/CAP394/WholeStory-Iris.html).

Fonte dos dados: [Iris Data Set](http://archive.ics.uci.edu/ml/datasets/Iris)

* Atributos:
    1. sepal length in cm (comprimento da sepal);
    2. sepal width in cm (largura da sepal);
    3. petal length in cm (comprimento da pétala);
    4. petal width in cm  (largura da pétala).

[**Python**] - Carregar h2o:

In [ ]:
import h2o
from h2o.automl import H2OAutoML

[**Python**] - Inicializar o h2o:

In [ ]:
h2o.init()

[**Python**] - Carregar os dados

In [ ]:
Iris= h2o.import_file('https://raw.githubusercontent.com/MathMachado/DataFrames/master/Iris.csv')
Iris.summary()

Este é um problema de classificação. Portanto, devemos definir a variável-target como uma classe.

[**Python**] - Definir a variável-target como classe:

In [ ]:
Iris['Species']= Iris['Species'].asfactor()

[**Python**] - Definir as amostras de treinamento e validação dos modelos preditivos:

In [ ]:
Iris_Treinamento, Iris_Teste= Iris.split_frame(ratios=[0.9])

[**Python**] - Mostrar os tamanhos das amostras de validação e treinamento:

In [ ]:
print(Iris_Treinamento.shape, Iris_Teste.shape)

[**Python**] - Definir as variáveis preditoras e target:

In [ ]:
preditoras= list(Iris_Treinamento.columns)
target= "Species"
preditoras.remove(target)
preditoras.remove('\ufeffId')
preditoras

[**Python**] - Configuração do AutoML:

In [ ]:
h2o_Iris= H2OAutoML(max_models= 10, max_runtime_secs = 300, seed= 20111974)
h2o_Iris.train(x= preditoras, y= target, training_frame= Iris_Treinamento)

[**Python**] - Listar os modelos candidatos:

In [ ]:
Leader_Board= h2o_Iris.leaderboard
Leader_Board.head()

[**Python**] - Mostrar a performance do modelo na amostra de validação

In [ ]:
h2o_Iris.leader

[**Python**] - Fazer predições/estimativas com o modelo campeão:

In [ ]:
Y_Predito= h2o_Iris.leader.predict(Iris_Teste)
Y_Predito

[**Python**] - Compare os resultados acima com os valores reais do dataframe:

In [ ]:
Iris_Teste

[**Python**] - Mostrar todos os parâmetros do modelo campeão:

In [ ]:
ID_Modelo_Campeao= h2o_Iris.leader.model_id
ID_Modelo_Campeao

In [ ]:
Modelo_Parametros= h2o.get_model(ID_Modelo_Campeao)
Modelo_Parametros.params

[**Python**] - Desligar o cluster h2o:

In [ ]:
h2o.cluster().shutdown()